In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import tensorflow as tf
import random

(train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()
nb_classes = 10


11493376/11490434 [==============================] - 0s 0us/step


In [3]:
train_y[0].shape, train_x[0].shape

((), (28, 28))

In [0]:
train_x = train_x.reshape([train_x.shape[0], -1])
train_x = train_x.astype('float32') / 255
# Label
train_y = tf.keras.utils.to_categorical(train_y, num_classes=nb_classes)

test_x = test_x.reshape([test_x.shape[0], -1])
test_x = test_x.astype('float32') / 255
# Label
test_y = tf.keras.utils.to_categorical(test_y, num_classes=nb_classes)

In [5]:
train_x.shape

(60000, 784)

In [6]:
nTrain = train_x.shape[0]
print(nTrain)

60000


In [0]:
buffer_size = train_x.shape[0] + test_x.shape[0]
MINIBATCH_SIZE = 100
dataset = tf.data.Dataset.from_tensor_slices(({"image": train_x}, train_y))
dataset = dataset.shuffle(buffer_size).repeat().batch(MINIBATCH_SIZE)
iterator = dataset.make_one_shot_iterator()
next_batch = iterator.get_next()

In [8]:
# MNIST data image of shape 28 * 28 = 784
X = tf.placeholder(tf.float32, [None, 784])
# 0 - 9 digits recognition = 10 classes
Y = tf.placeholder(tf.float32, [None, nb_classes])

W = tf.Variable(tf.random_normal([784, nb_classes]))
b = tf.Variable(tf.random_normal([nb_classes]))

#x = tf.placeholder(tf.float32, [None, 784])

# Hypothesis (using softmax)
logit=tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logit)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logit, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)
#optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
# Test model
is_correct = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))

# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [10]:
training_epochs = 15
batch_size = 100

with tf.Session() as sess:

    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(nTrain / batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = sess.run(next_batch)
            c, _ = sess.run([cost, optimizer], feed_dict={
                            X: batch_xs['image'], Y: batch_ys})
            avg_cost += c / total_batch
        print('Epoch:', '%04d' % (epoch + 1),
              'cost =', '{:.9f}'.format(avg_cost))
    print("Learning finished")

    # Test the model using test sets
    print("Accuracy: ", accuracy.eval(session=sess, feed_dict={
          X: test_x, Y: test_y}))

    # Get one and predict
    r = random.randint(0, test_x.shape[0] - 1)
    print("Label: ", sess.run(tf.argmax(test_y[r:r + 1], 1)))
    print("Prediction: ", sess.run(
        tf.argmax(hypothesis, 1), feed_dict={X: test_x[r:r + 1]}))

Epoch: 0001 cost = 2.956580084
Epoch: 0002 cost = 1.061415347
Epoch: 0003 cost = 0.848653309
Epoch: 0004 cost = 0.744805501
Epoch: 0005 cost = 0.677839160
Epoch: 0006 cost = 0.630148680
Epoch: 0007 cost = 0.593393266
Epoch: 0008 cost = 0.565040014
Epoch: 0009 cost = 0.541103795
Epoch: 0010 cost = 0.521311258
Epoch: 0011 cost = 0.504333811
Epoch: 0012 cost = 0.489194048
Epoch: 0013 cost = 0.475987515
Epoch: 0014 cost = 0.464920177
Epoch: 0015 cost = 0.454278611
Learning finished
Accuracy:  0.891
Label:  [3]
Prediction:  [3]
